# Imports and general formatting

In [15]:
import pandas as pd
import numpy as np
import copy
from collections import Counter

In [103]:
from scripts.QC.GT_matrices import Genotypes, Pedigree
from scripts.downstream_formatting import *

#### load genotypes:

In [6]:
genotype_path = "various_subsets/raw_geno_all_samples_fill3mbgaps.csv"

In [9]:
genotypes = pd.read_csv(genotype_path)
genotypes.index = genotypes["Unnamed: 0"].astype(str)
del(genotypes["Unnamed: 0"])

#### load phenotypes:

In [10]:
phenotype_path = "/home/tilman/nas/AIL_phenotypes20190826.csv"

In [12]:
phenotypes = pd.read_csv(phenotype_path, sep=",")

In [106]:
pedigree_file = "/home/tilman/nas/pedigree/AIL_pedigree_20190826.tsv"

In [140]:
def load_pedigree(pedfile):
    """Returns a dictionary from a input file """
    raw_ped = pd.read_csv(pedfile, sep="\t")
    raw_ped.index=raw_ped["ID"]
    ped_d = {}
    for i, k in raw_ped.iterrows():
        ped_d[str(i)] = {"sire":str(k["Sire"]).split(".")[0],
                    "dam":str(k["Dam"]).split(".")[0],
                    "sex":str(k["Sex"]),
                    "generation":str(i)[-2:]}
    return ped_d

In [141]:
p = load_pedigree(pedfile=pedigree_file)

In [111]:
pedigree = pd.read_csv("/home/tilman/nas/pedigree/AIL_pedigree_20190826.tsv", sep="\t")

In [129]:
pedigree_obj = Pedigree(pedigree_file=pedigree_file)

In [144]:
pedigree_obj.pedigree=p

In [113]:
pedigree.index = pedigree.ID
phenotypes.index = phenotypes.ID

##### transfer some sex data to the phenotype file.

In [29]:
new_sex = []
for i,k in phenotypes.iterrows():
    if float(k["SEX"]) not in [1.0,2.0]:
        try:
            if float(pedigree.loc[i]["Sex"]) in [1.0,2.0]:
                new_sex.append(float(pedigree.loc[i]["Sex"]))
            else:
                new_sex.append(float(k["SEX"]))
        except KeyError:
            print("{} not in pedigree".format(i))
            new_sex.append(float(k["SEX"]))
    else:
        new_sex.append(float(k["SEX"]))

225109 not in pedigree
232509 not in pedigree
543608 not in pedigree


In [40]:
print(Counter(phenotypes.SEX).most_common(5))
print(Counter(new_sex).most_common(5))
print((2277+2059)-(1828+1618)) #recovered sex data for 890 samples, nice

[(2.0, 1828), (1.0, 1618), (nan, 1), (nan, 1), (nan, 1)]
[(2.0, 2277), (1.0, 2059), (nan, 1), (nan, 1), (nan, 1)]
890


In [41]:
phenotypes["SEX"]=new_sex

### filter out bad phenotypes

In [45]:
phenotypes["ID"] = phenotypes["ID"].astype(str)

In [46]:
generation = [i[-2:] for i in phenotypes["ID"]]

In [51]:
phenotypes.GENERATION = generation

In [82]:
notes = []
new_bw8 = []
for i, k in phenotypes.iterrows():
    try:
        float(k["BW8"])
        notes.append("")
        new_bw8.append(float(k["BW8"]))
    except ValueError:
        print(k["BW8"])
        notes.append(k["BW8"])
        new_bw8.append(np.nan)

died D33
died D47
Died D31
Died D40
Died D30
Died D35
died D48
Died D36
died D41
Died D41
died D35
died D30
Died D43
died D49


In [84]:
print(len(new_bw8))

4485


In [85]:
phenotypes["NOTES"]=notes
phenotypes["BW8"]=new_bw8

### make normalised-by-generation BW8 phenotypes.

$ \frac{BW8-mean(BW8)}{var(BW8)}$

In [98]:
var_mean_d = {}
for i in set(phenotypes.GENERATION):
    #print(i)
    var_mean_d[i]={}
    
    curr_gen = phenotypes.loc[phenotypes.GENERATION == i]
    var = np.var(curr_gen["BW8"].astype(float))
    #print(var)
    mean = curr_gen["BW8"].astype(float).mean(skipna=True)
    var_mean_d[i]["variance"] = var
    var_mean_d[i]["mean"] = mean
    #print(curr_gen["BW8"].astype(float).mean(skipna=True))

In [94]:
norm = []
for i,k in phenotypes.iterrows():
    var = var_mean_d[k["GENERATION"]]["variance"]
    mean = var_mean_d[k["GENERATION"]]["mean"]
    normval = (k["BW8"]-mean)/var
    #print(i, k["BW8"], var, mean)
    #print("#########")
    norm.append(normval)

In [97]:
phenotypes["BW8_norm_by_gen"] = norm

### add Dam and Sire so i can control for family

In [149]:
dams = []
sires = []
for i, k in phenotypes.iterrows():
    try:
        dam, sire = pedigree_obj.get_parents(sample=str(i))
        #print(dam,sire)
        dams.append(dam)
        sires.append(sire)
    except KeyError:
        #print("{} is not in the pedigree".format(i))
        dams.append(np.nan)
        sires.append(np.nan)

In [153]:
phenotypes["DAM"]=dams
phenotypes["SIRE"]=sires

### export phenotypes again

In [156]:
phenotypes.to_csv("/home/tilman/nas/AIL_phenotypes20190827.csv")
phenotypes.to_csv("/home/tilman/nas/AIL_phenotypes20190827.tsv", sep="\t")

phenotypes.to_csv("/mnt/bbg/AIL_reseq/useful_info/phenotypes/AIL_phenotypes20190827.csv")
phenotypes.to_csv("/mnt/bbg/AIL_reseq/useful_info/phenotypes/AIL_phenotypes20190827.tsv", sep="\t")

### make subset for easy rqtl input generation:

In [172]:
nss = phenotypes[["ID","BW8", "BW8_norm_by_gen", "SEX", "GENERATION", "DAM", "SIRE"]]

In [174]:
nss = copy.deepcopy(nss) # no messing with the original dataframe, tilman

In [177]:
nss.columns = ['id', 'BW8', 'BW8_norm_by_gen', 'SEX', 'GENERATION', 'DAM', 'SIRE']

# Minimally filtered

## Make all_samples input

In [178]:
make_rqtl_input(gen_out="20190827_rqtl_all_geno.csv", phe_out="20190827_rqtl_all_pheno.csv", old_id_pheno=False, df=genotypes, phe_id_col="id", phenotype=nss)

## Make F2_samples input

In [180]:
F2s = nss.loc[nss["GENERATION"]=="02"]

In [181]:
make_rqtl_input(gen_out="20190827_rqtl_f2_geno.csv", phe_out="20190827_rqtl_f2_pheno.csv", old_id_pheno=False, df=genotypes, phe_id_col="id", phenotype=F2s)

/home/tilman/nas/chicken_finemapping/QTL/stripes2rqtl_multigen/scripts/downstream_formatting.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  phenotypes_raw[phe_id_col] = phenotypes_raw[phe_id_col].astype(str)


## Make F3_samples input

In [182]:
F3s = nss.loc[nss["GENERATION"]=="03"]

In [183]:
make_rqtl_input(gen_out="20190827_rqtl_f3_geno.csv", phe_out="20190827_rqtl_f3_pheno.csv", old_id_pheno=False, df=genotypes, phe_id_col="id", phenotype=F3s)

### make F8_samples input

In [184]:
F8s = nss.loc[nss["GENERATION"]=="08"]

In [185]:
make_rqtl_input(gen_out="20190827_rqtl_f8_geno.csv", phe_out="20190827_rqtl_f8_pheno.csv", old_id_pheno=False, df=genotypes, phe_id_col="id", phenotype=F8s)

## Make F15_samples input

In [186]:
F15s = nss.loc[nss["GENERATION"]=="15"]

In [187]:
make_rqtl_input(gen_out="20190827_rqtl_f15_geno.csv", phe_out="20190827_rqtl_f15_pheno.csv", old_id_pheno=False, df=genotypes, phe_id_col="id", phenotype=F15s)